<a target="_blank" href="https://colab.research.google.com/github/EasyScience/EasyDiffractionLib/blob/master/examples/Fitting_PD-NEUT-TOF_NCAF-WISH.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

# Fitting `PD-NEUT-TOF` NCAF-WISH

This example shows how to refine the crystal structure parameters of Si from neutron diffraction data in a time-of-flight experiment performed on SEPD diffractometer at Argonne.

## Import

In [ ]:
# Code related to Google Colab
import importlib.util
import os

import pooch

if os.getenv("COLAB_RELEASE_TAG"):
    print("Running in Google Colab")
    # Install the easydiffraction library if it is not installed (including charts extras)
    if importlib.util.find_spec("easydiffraction") is None:
        !pip install 'easydiffraction[charts]'
    # Download the data files to be read in notebook
    for fname in ['ncaf.cif', 'wish.xye']:
        pooch.retrieve(
            url=f"https://raw.githubusercontent.com/EasyScience/EasyDiffractionLib/refs/heads/master/examples/data/{fname}",
            known_hash=None,
            fname=fname,
            path="data",
        )

In [ ]:
import easydiffraction as ed

## Job

Create a job — the main object to store all the information

In [ ]:
job = ed.Job(type='tof')

## Model

Load a phase from CIF file

In [ ]:
job.add_phase_from_file('data/ncaf.cif')
print(job.phases)

Show phase info in CIF format

In [ ]:
phase = job.phases['ncaf']
print(phase.cif)

Display the crystal structure of a given model

In [ ]:
job.show_crystal_structure(id='ncaf')

## Experiment

Load experimentally measured data from a file in XYE format

In [ ]:
job.add_experiment_from_file('data/wish.xye')

Display the experimentally measured data

In [ ]:
job.show_experiment_chart()

Define a point background

In [ ]:
background_points = [
    (  9162, 465),
    ( 11136, 593),
    ( 13313, 497),
    ( 14906, 546),
    ( 16454, 533),
    ( 17352, 496),
    ( 18743, 428),
    ( 20179, 452),
    ( 21368, 397),
    ( 22176, 468),
    ( 22827, 477),
    ( 24644, 380),
    ( 26439, 381),
    ( 28257, 378),
    ( 31196, 343),
    ( 34034, 328),
    ( 37265, 310),
    ( 41214, 323),
    ( 44827, 283),
    ( 49830, 273),
    ( 52905, 257),
    ( 58204, 260),
    ( 62916, 261),
    ( 70186, 262),
    ( 74204, 262),
    ( 82103, 268),
    ( 91958, 268),
    (102712, 262)
]
job.set_background(background_points)

Display the experiment chart after setting the background

In [ ]:
job.show_experiment_chart()

## Analysis

Display the analysis chart before setting initial parameter values

In [ ]:
job.show_analysis_chart()

Create aliases for the two types of experimental parameters

In [ ]:
pattern= job.pattern
instrument = job.instrument

Change the default value of some instrumental parameters and display the analysis chart again

In [ ]:
instrument.dtt1 = 20770
instrument.dtt2 = -1.08308
instrument.ttheta_bank = 152.827
job.show_analysis_chart()

Change the scale and display the analysis chart again

In [ ]:
phase.scale = 0.5
job.show_analysis_chart()

Change the default values of the peak profile related parameters and display the analysis chart again

In [ ]:
instrument.alpha0 = 0
instrument.alpha1 = 0.1
instrument.beta0 = 0.01
instrument.beta1 = 0.01
instrument.sigma0 = 0
instrument.sigma1 = 0
instrument.sigma2 = 5
job.show_analysis_chart()

Select parameters to be refined in the first round

In [ ]:
phase.scale.free = True
pattern.zero_shift.free = True

Print parameters to be refined (free parameters) before fitting

In [ ]:
job.show_free_parameters()

Start Least-Squares minimization to refine the selected parameters

In [ ]:
job.fit()

Print the refined parameters after fitting

In [ ]:
job.show_free_parameters()

Display the analysis chart after the first fitting

In [ ]:
job.show_analysis_chart()

Select more parameters to be refined in the second round

In [ ]:
instrument.alpha1.free = True
instrument.beta0.free = True
instrument.beta1.free = True
instrument.sigma2.free = True

Print free parameters before the second fitting

In [ ]:
job.show_free_parameters()

Start the second round of minimization

In [ ]:
job.fit()

Print free parameters after the third fitting

In [ ]:
job.show_free_parameters()

Display the analysis chart after the third fitting

In [ ]:
job.show_analysis_chart()

Show all parameters after refinement

In [ ]:
job.show_parameters()